# Day 4 - Finite Markov Decision Processes

## The Agent-Environment Interface

* A Markov decision process is defined by an $environment$ and an $agent$
* The environment, at each time step $t$, gives the agent the next $state$ and $reward$
* The agent then chooses an $action$ to perform, which influences what state the environment transitions to
* This formalism is flexible and general:
    - Time steps can be arbitrary intervals
    - Actions can be low-level or high-level
    - States can be concrete sensor readings or abstract, conceptual mental states
    - Actions could even be mental/computational

### $Exercise\ \mathcal{3.1}$

#### Devise three example tasks of your own that fit into the MDP framework, identifying for each its states, actions, and rewards. Make the three examples as different from each other as possible. The framework is abstract and flexible and can be applied in many different ways. Stretch its limits in some way in at least one of your examples.

1. Playing Minecraft:
    - The states are the frames of the game, or more accurately some learned, lower-dimensional representation of them.
    - Actions are keyboard button presses and mouse movements.
    - The agent could receive a reward each time it unlocks an Advancement in the game
        * As these are very sparse rewards, perhaps rewards could be given every time the agent is surprised in some way, to encourage exploration     
1. Cooking pasta:
    - The states are information about the level to which the pot is filled with water, the salt content of the water, as well as its temperature
    - Actions are filling the pot with water, pouring some water out, adding salt, turning up/down the heat, putting the pasta in the pot, and signalling when the pasta is considered "done."
    - The rewards could come in the form of ratings by humans tasting the pasta
1. A robot learning to do what a human wants:
    - The states would simply be the robot's sensor readings, perhaps compressed into learned representations
    - Actions would be voltages applied to the robot's motors
    - Rewards come in the form of the human pressing a reward button whenever they're pleased with what the robot did

### $Exercise\ \mathcal{3.2}$

#### Is the MDP framework adequate to usefully represent all goal-directed learning tasks? Can you think of any clear exceptions?

One task that may be difficult to represent as an MDP is to "make scientific progress." It seems quite difficult to define a reward function that somehow represents this goal. Though, if a learning algorithm is sufficiently powerful and sample-efficient, it might be possible to simply use a manual reward signal from a human, given whenever they make the subjective judgment that the agent is moving in the direction of progress.

### $Exercise\ \mathcal{3.3}$

#### Consider the problem of driving. You could define the actions in terms of the accelerator, steering wheel, and brake, that is, where your body meets the machine. Or you could define them farther out—say, where the rubber meets the road, considering your actions to be tire torques. Or you could define them farther in—say, where your brain meets your body, the actions being muscle twitches to control your limbs. Or you could go to a really high level and say that your actions are your choices of where to drive. What is the right level, the right place to draw the line between agent and environment? On what basis is one location of the line to be preferred over another? Is there any fundamental reason for preferring one location over another, or is it a free choice?

Where to draw this line depends on how well we can control the car, and on how well our agent can actually learn in practice. A vehicle that is already able to drive autonomously, or perhaps simply has a human driver, can still be part of a reinforcement learning problem, where the actions are at the highest level of deciding where to drive. This could be to optimize travel time, for example.  
Meanwhile, when the task is to teach a physical robot how to drive a car, and the robot is supposed to also learn other tasks, then, if we have a sufficiently powerful learning algorithm, the lowest level seems like the best choice. Enabling the robot to learn how to move its limbs will give it a more general skill that transfers to different tasks more seamlessly. It is also possible to train a second agent to make the higher-level decisions that guidet he robotic agent's actions.

### $Exercise\ \mathcal{3.4}$

#### Give a table analogous to that in Example 3.3, but for $p(s',r|s,a)$. It should have columns for $s,\ a,\ s',\ r,\ $and $p(s',r|s,a)$, and a row for every 4-tuple for which $p(s',r|s,a) > 0$.

|$s$ |$a$     |$s'$|$r$         |$p(s',r|s,a)$|
|----|--------|----|------------|-------------|
|high|search  |high|$r_{search}$|$\alpha$     |
|high|search  |low |$r_{search}$|$1-\alpha$   |
|high|wait    |high|$r_{wait}$  |1            |
|low |search  |high|$-3$        |$1-\beta$    |
|low |search  |low |$r_{search}$|$\beta$      |
|low |wait    |low |$r_{wait}$  |1            |
|low |recharge|high|$0$         |1            |

## Goals and Rewards

* The $reward\ hypothesis$ states that all goals can be thought of as the maximization of the expected cumulative reward

## Returns and Episodes

* The return from time step $t$, $G_t$, is defined as the sum of future rewards
* For continuing tasks, the $discounted\ return\;G_t=R_{t+1}+\gamma R_{t+2}+\gamma^2R_{t+3}+\dots=\sum_{k=0}^\infty \gamma^kR_{t+k+1}$ is used
* The goal of the agent is to maximize the expectation of $G_t$
* The discouned reward can be written recursively as $G_t=R_{t+1}+\gamma G_{t+1}$

### $Exercise\ \mathcal{3.5}$

#### The equations in Section 3.1 are for the continuing case and need to be modified (very slightly) to apply to episodic tasks. Show that you know the modifications needed by giving the modified version of (3.3).

The equation does not take into account that a transition into the terminal state exists, so we need to sum over $s'\in\mathcal{S}^+$:
$$
\sum_{s'\in \mathcal{S}^+}\sum_{r\in\mathcal{R}}p(s',r|s,a)=1,\ \text{for all }s\in\mathcal{S}, a\in\mathcal{A}(s)
$$

### $Exercise\ \mathcal{3.6}$

#### Suppose you treated pole-balancing as an episodic task but also used discounting, with all rewards zero except for $-1$ upon failure. What then would the return be at each time? How does this return differ from that in the discounted, continuing formulation of this task?

The return in this case would simply be $-\gamma^{K-1}$, where $K$ is again the number of time steps before failure. The difference to the continuing case is that this is the only nonzero reward received, and the return is not a sum of the rewards of all expected future failures.

### $Exercise\ \mathcal{3.7}$

#### Imagine that you are designing a robot to run a maze. You decide to give it a reward of $+1$ for escaping from the maze and a reward of zero at all other times. The task seems to break down naturally into episodes—the successive runs through the maze—so you decide to treat it as an episodic task, where the goal is to maximize expected total reward (3.7). After running the learning agent for a while, you find that it is showing no improvement in escaping from the maze. What is going wrong? Have you effectively communicated to the agent what you want it to achieve?

If the reward is always $1$ for escaping, undiscounted, then there is no difference between solving the maze as fast as possible, and taking several decades to solve it. By discounting the reward, or providing a small negative reward on each time step, this problem would be solved.

### $Exercise\ \mathcal{3.8}$

#### Suppose  $\gamma=0.5$ and the following sequence of rewards is received $R_1=-1,\ R_2=2,\ R_3=6,\ R_4=3,\ \text{and}\ R_5=2$, with $T=5$. What are $G_0,G_1,\dots,G_5$? Hint: Work backwards.

$$
\begin{align}
G_5&=0 \\
G_4&=R_5+\gamma G_5=2 \\
G_3&=R_4+\gamma G_4=4 \\
G_2&=R_3+\gamma G_3=8 \\
G_1&=R_2+\gamma G_2=6 \\
G_0&=R_1+\gamma G_1=2
\end{align}
$$

### $Exercise\ \mathcal{3.9}$

#### Suppose $\gamma=0.9$ and the reward sequence is $R_1=2$ followed by an infinite sequence of $7$s. What are $G_1$ and $G_0$?

$$
\begin{align}
G_1&=7\sum_{k=0}^\infty\gamma^k=\frac{7}{1-\gamma}=70 \\
G_0&=2+\gamma G_1=65
\end{align}
$$

### $Exercise\ \mathcal{3.10}$

#### Prove the second equality in (3.10).

First, we multiply both sides by $\gamma$, which gives us
$$
\gamma G_t=\gamma\sum_{k=0}^\infty\gamma^k=\sum_{k=0}^\infty\gamma^{k+1}=\sum_{k'=1}^\infty\gamma^{k'},
$$
which lets us do
$$
\begin{align}
G_t-\gamma G_t&=\quad\ \ \sum_{k=0}^\infty\gamma^{k}-\sum_{k'=1}^\infty\gamma^{k'} \\
&=1+\sum_{k=1}^\infty\gamma^k-\sum_{k'=1}^\infty\gamma^{k'} \\
&=1.
\end{align}
$$
Finally, upon rearranging, we find that
$$
G_t=\frac{1}{1-\gamma}.
$$

## Unified Notation for Episodic and Continuing Tasks

* To unify both cases, we can introduce an $absorbing\ state$ to turn episodic tasks into continuing tasks
* Instead of an episode ending once the terminal state is reached, the absorbing state is reached, which transitions only to itself, giving a reward of 0
* Setting $\gamma=1$ now gives us the same infinite sum for the return as for the continuing case
* Alternatively, we can use the unified notation

$$
G_t\doteq\sum_{k=t+1}^T\gamma^{k-t-1}R_k
$$
* Here, either $\gamma=1$, deonting the episodic case, or $T=\infty$, denoting the continuing case, but never both

## Policies and Value Functions

* Almost all RL algorithms try to learn $value\ functions$
* The value function of a state is the expected return from that state
* A $policy$ is a function mapping from states to the probabilities of taking each action from that state
* For all $s\in\mathcal{S}$,

$$
v_\pi(s)\doteq\mathbb E_\pi\left[G_t|S_t=s\right]=\mathbb E_\pi\left[\sum_{k=0}^\infty\gamma^kR_{t+k+1}\Biggr|S_t=s\right]
$$
* For $action-values$,

$$
q_\pi(s,a)\doteq\mathbb E_\pi\left[G_t|S_t=s,A_t=a\right]=\mathbb E_\pi\left[\sum_{k=0}^\infty\gamma^kR_{t+k+1}\Biggr|S_t=s,A_t=a\right]
$$
* These can be estimated from experience
* The $Bellman\ equations$ show the value functions' recursive nature:

$$
\begin{align}
v_\pi(s)&\doteq\mathbb E_\pi[G_t|S_t=s] \\
&=\mathbb E_\pi[R_{t+1}+\gamma G_{t+1}|S_t=s] \\
&=\sum_a\pi(a|s)\sum_{s'}\sum_rp(s',r|s,a)\left[r+\gamma\mathbb E_\pi[G_{t+1}|S_{t+1}=s']\right] \\
&=\sum_a\pi(a|s)\sum_{s',r}p(s',r|s,a)\left[r+\gamma v_\pi(s')\right]
\end{align}
$$

### $Exercise\ \mathcal{3.11}$

#### If the current state is $S_t$, and actions are selected according to a stochastic policy $\pi$, then what is the expectation of $R_{t+1}$ in terms of $\pi$ and the four-argument function $p$ (3.2)?

$$
\mathbb E[R_{t+1}]=\sum_a\pi(a|S_t)\sum_{s',r}p(s',r|S_t,a)r
$$

### $Exercise\ \mathcal{3.12}$

#### Give an equation for $v_\pi$ in terms of $q_\pi$ and $\pi$.

$$
v_\pi(s)=\sum_a\pi(a|s)q_\pi(s,a)
$$

### $Exercise\ \mathcal{3.13}$

#### Give an equation for $q_\pi$ in terms of $v_\pi$ and the four-argument $p$.

$$
q_\pi(s,a)=\sum_{s',r}p(s',r|s,a)\left[r+\gamma v_\pi(s')\right]
$$

### $Exercise\ \mathcal{3.14}$

#### The Bellman equation (3.14) must hold for each state for the value function $v_\pi$ shown in Figure 3.2 (right) of Example 3.5. Show numerically that this equation holds for the center state, valued at $+0.7$, with respect to its four neighboring states, valued at $+2.3$, $+0.4$, $-0.4$, and $+0.7$. (These numbers are accurate only to one decimal place.)

$$
v_\pi(center)=0+0.9\cdot\frac{2.3-0.4+0.4+0.7}{4}=\frac{9}{10}\cdot\frac{3}{4}\approx0.7
$$

### $Exercise\ \mathcal{3.15}$

#### In the gridworld example, rewards are positive for goals, negative for running into the edge of the world, and zero the rest of the time. Are the signs of these rewards important, or only the intervals between them? Prove, using (3.8), that adding a constant $c$ to all the rewards adds a constant, $v_c$, to the values of all states, and thus does not affect the relative values of any states under any policies. What is $v_c$ in terms of $c$ and $\gamma$?

If $G_t'$ is the return received after adding a constant $c$ to each reward, then
$$
\begin{align}
G_t'&\doteq(R_{t+1}+c)+\gamma(R_{t+2}+c)+\gamma^2(R_{t+3}+c)+\dots \\
&=\sum_{k=0}^\infty\gamma^k(R_{t+k+1}+c) \\
&=\sum_{k=0}^\infty\gamma^kR_{t+k+1}+\sum_{k=0}^\infty\gamma^kc \\
&=G_t+c\sum_{k=0}^\infty\gamma^k \\
&=G_t+\frac{c}{1-\gamma}.
\end{align}
$$
We can now define the adjusted value to be
$$
\begin{align}
v_\pi'(s)&\doteq\mathbb E\left[G_t'|S_t=s\right] \\
&=\mathbb E\left[G_t+\frac{c}{1-\gamma}\Biggr|S_t=s\right] \\
&=\mathbb E\left[G_t|S_t=s\right]+\frac{c}{1-\gamma} \\
&=v_\pi(s)+\frac{c}{1-\gamma}.
\end{align}
$$
Thus, we can define $$v_c\doteq\frac{c}{1-\gamma}.$$

### $Exercise\ \mathcal{3.16}$

#### Now consider adding a constant $c$ to all the rewards in an episodic task, such as maze running. Would this have any effect, or would it leave the task unchanged as in the continuing task above? Why or why not? Give an example.

In the undiscounted episodic case, adding a constant $c$ to each reward will increase the return of an episode by $cT$. If $c$ is negative, this encourages shorter episode length, while a sufficiently large $c$ can make some nonterminal state transition rewards positive, giving the agent an incentive to get into a loop to acquire infinite reward.  
In the maze example, where each reward is $-1$, the agent is usually incentivized to solve the maze as quickly as possible. For any $c>1$, the agent can now accumulate infinite reward by staying in the maze forever.

### $Exercise\ \mathcal{3.17}$

#### What is the Bellman equation for action values, that is, for $q_\pi$? It must give the action value $q_\pi(s,a)$ in terms of the action values, $q_\pi(s',a')$, of possible successors to the state–action pair $(s,a)$. Hint: The backup diagram to the right corresponds to this equation. Show the sequence of equations analogous to (3.14), but for action values.

$$
\begin{align}
q_\pi(s,a)&\doteq\mathbb E\left[G_t|S_t=s,A_t=a\right] \\
&=\mathbb E\left[R_{t+1}+\gamma G_{t+1}|S_t=s,A_t=a\right] \\
&=\sum_{s'}\sum_rp(s',r|s,a)\left[r+\gamma\sum_{a'}\pi(a'|s')\mathbb E\left[G_{t+1}|S_{t+1}=s',A_{t+1}=a'\right]\right] \\
&=\sum_{s',r}p(s',r|s,a)\left[r+\gamma\sum_{a'}\pi(a'|s')q_\pi(s',a')\right]
\end{align}
$$

### $Exercise\ \mathcal{3.18}$

#### The value of a state depends on the values of the actions possible in that state and on how likely each action is to be taken under the current policy. We can think of this in terms of a small backup diagram rooted at the state and considering each possible action:

$$
\text{(See book)}
$$
#### Give the equation corresponding to this intuition and diagram for the value at the root node, $v_\pi(s)$, in terms of the value at the expected leaf node, $q_\pi(s,a)$, given $S_t=s$. This equation should include an expectation conditioned on following the policy, $\pi$.

$$
v_\pi(s)=\mathbb E_{A_t\sim\pi}\left[q_\pi(S_t,A_t)|S_t=s\right]
$$

#### Then give a second equation in which the expected value is written out explicitly in terms of $\pi(a|s)$ such that no expected value notation appears in the equation.

$$
v_\pi(s)=\sum_a\pi(a|s)q_\pi(s,a)
$$

### $Exercise\ \mathcal{3.19}$

#### The value of an action, $q_\pi(s,a)$, depends on the expected next reward and the expected sum of the remaining rewards. Again we can think of this in terms of a small backup diagram, this one rooted at an action (state–action pair) and branching to the possible next states:

$$
\text{(See book)}
$$

#### Give the equation corresponding to this intuition and diagram for the action value, $q_\pi(s,a)$, in terms of the expected next reward, $R_{t+1}$, and the expected next state value, $v_\pi(S_{t+1})$, given that $S_t=s$ and $A_t=a$. This equation should include an expectation but not one conditioned on following the policy.

$$
q_\pi(s,a)=\mathbb E_{S_{t+1},R_{t+1}\sim p}\left[R_{t+1}+\gamma v_\pi(S_{t+1})|S_t=s,A_t=a\right]
$$

#### Then give a second equation, writing out the expected value explicitly in terms of $p(s',r|s,a)$ defined by (3.2), such that no expected value notation appears in the equation.

$$
q_\pi(s,a)=\sum_{s',r}p(s',r|s,a)\left[r+\gamma v_\pi(s')\right]
$$

## Optimal Policies and Value Functions

* One policy is better than another if the value of *all* states under that policy are larger than the values under the other policy
* There is always at least one policy that is better than, or equal to all other policies, called an $optimal\ policy\ \pi_*$
* For these policies, there are the optimal state value function $v_*(s)$, and the optimal action value function $q_*(s,a)$
* The Bellman equations for these are called the $Bellman\ optimality\ equations$
* Once $v_*$ is known, any policy that is greedy with respect to a one-step search is an optimal policy
* Once $q_*$ is known, an optimal policy simply has to choose the action $a$ that maximizes this
* This allows optimal action selection without knowing anything about the environment's dynamics

### $Exercise\ \mathcal{3.20}$

#### Draw or describe the optimal state-value function for the golf example.

The optimal state-value function looks like a combination of the state- and action-value functions shown in the book. Anywhere outside of the green, it looks like the optimal action-value function for the driver, as that is the optimal action in those states, while on the green, it looks like the putter state-value function, as that is the optimal action anywhere on the green.

### $Exercise\ \mathcal{3.21}$

#### Draw or describe the contours of the optimal action-value function for putting, $q_*(s, putter)$, for the golf example.

Anywhere on the green, as well as anywhere the green can be reached in one or two strokes by the putter, this looks like the putter state value function in the book. Further out, it looks like the driver action-value function, with each value reduced by $1$, as one stroke is essentially wasted on a putt. This reduction by $1$ does not apply to states where one putt reduces the number of driver strokes required to reach the green, as choosing the driver would not reduce the number of strokes required.

### $Exercise\ \mathcal{3.22}$

#### Consider the continuing MDP shown to the right. The only decision to be made is that in the top state, where two actions are available, left and right. The numbers show the rewards that are received deterministically after each action. There are exactly two deterministic policies, $\pi_{left}$ and $\pi_{right}$.

Naming the top, left, and right states $t$, $l$, and $r$ respectively, we get the following:
$$
\begin{align}
v_{\pi_{left}}(t)&=1+\gamma0+\gamma^21+\gamma^30+\dots=\sum_{k=0}^\infty(\gamma^2)^k&&=\frac{1}{1-\gamma^2} \\
v_{\pi_{right}}(t)&=0+\gamma2+\gamma^20+\gamma^32+\dots=2\gamma\sum_{k=0}^\infty(\gamma^2)^k=\frac{2\gamma}{1-\gamma^2}&&=2\gamma v_{\pi_{left}}(t) \\
v_{\pi_{left}}(l)&=0+\gamma v_{\pi_{left}}(t)&&=\frac{\gamma}{1-\gamma^2} \\
v_{\pi_{right}}(l)&=0+\gamma v_{\pi_{right}}(t)=\frac{2\gamma^2}{1-\gamma^2}&&=2\gamma v_{\pi_{left}}(l) \\
v_{\pi_{left}}(r)&=2+\gamma v_{\pi_{left}}(t)&&=2+\frac{\gamma}{1-\gamma^2} \\
v_{\pi_{right}}(r)&=2+\gamma v_{\pi_{right}}(t)=2+\frac{2\gamma^2}{1-\gamma^2}&&=2+2\gamma(v_{\pi_{left}}(r)-2)
\end{align}
$$

#### What policy is optimal if $\gamma=0$?

In this case, both policies have the same value in states $l$ and $r$, but $\pi_{left}$ has a value of $1$ in state $t$, while $\pi_{right}$ has a value of $0$ there, making $\pi_{left}$ strictly superior.

#### If $\gamma=0.9$?

In each state, as $2\gamma=1.8>1$, $\pi_{right}$'s value is superior.

#### If $\gamma=0.5$?

Here, $2\gamma=1$, so the values of both policies are the same in all states.

### $Exercise\ \mathcal{3.23}$

#### Give the Bellman equation for $q_*$ for the recycling robot.

$$
\begin{align}
q_*(\mathtt{h},\mathtt{\;\ s})&=\alpha\left(r_{search}+\gamma\underset{a'}{\operatorname{max}}q_*(\mathtt{h},a)\right)+(1-\alpha)\left(r_{search}+\gamma\underset{a'}{\operatorname{max}}q_*(\mathtt{l},a)\right) \\
q_*(\mathtt{h},\mathtt{\;\ w})&=r_{wait}+\gamma\underset{a'}{\operatorname{max}}q_*(\mathtt{h},a) \\
q_*(\mathtt{l},\mathtt{\;\ s})&=\beta\left(r_{search}+\gamma\underset{a'}{\operatorname{max}}q_*(\mathtt{l},a)\right)+(1-\beta)\left(-3+\gamma\underset{a'}{\operatorname{max}}q_*(\mathtt{h},a)\right) \\
q_*(\mathtt{l},\mathtt{\;\ w})&=r_{wait}+\gamma\underset{a'}{\operatorname{max}}q_*(\mathtt{l},a) \\
q_*(\mathtt{l},\mathtt{re})&=\gamma\underset{a'}{\operatorname{max}}q_*(\mathtt{h},a)
\end{align}
$$

### $Exercise\ \mathcal{3.24}$

#### Figure 3.5 gives the optimal value of the best state of the gridworld as 24.4, to one decimal place. Use your knowledge of the optimal policy and (3.8) to express this value symbolically, and then to compute it to three decimal places.

$$
v_*(A)=10+\gamma^510+\gamma^{10}10+\dots=\frac{10}{1-\gamma^{5}}\approx24.419
$$

### $Exercise\ \mathcal{3.25}$

#### Give an equation for $v_*$ in terms of $q_*$.

$$
v_*(s)=\underset{a}{\operatorname{max}}q_*(s,a)
$$

### $Exercise\ \mathcal{3.26}$

#### Give an equation for $q_*$ in terms of $v_*$ and the four-argument $p$.

$$
q_*(s,a)=\sum_{s',r}p(s',r|s,a)v_*(s')
$$

### $Exercise\ \mathcal{3.27}$

#### Give an equation for $\pi_*$ in terms of $q_*$.

$$
\pi_*(s)=\operatorname{arg}\underset{a}{\operatorname{max}}q_*(s,a)
$$

### $Exercise\ \mathcal{3.28}$

#### Give an equation for $\pi_*$ in terms of $v_*$ and the four-argument $p$.

$$
\pi_*(s)=\operatorname{arg}\underset{a}{\operatorname{max}}\sum_{s',r}p(s',r|s,a)v_*(s')
$$

### $Exercise\ \mathcal{3.29}$

#### Rewrite the four Bellman equations for the four value functions ($v_\pi$, $v_*$, $q_\pi$, and $q_*$) in terms of the three argument function $p$ (3.4) and the two-argument function $r$ (3.5).

$$
\begin{align}
v_\pi(s)&=\sum_a\pi(a|s)\sum_{s',r}p(s'|s,a)\left[r(s,a)+\gamma v_\pi(s')\right] \\
v_*(s)&=\underset{a}{\operatorname{max}}\sum_{s',r}p(s'|s,a)\left[r(s,a)+\gamma v_*(s')\right] \\
q_\pi(s,a)&=\sum_{s',r}p(s'|s,a)\left[r(s,a)+\gamma\sum_{a'}\pi(a'|s')q_\pi(s',a')\right] \\
q_*(s,a)&=\sum_{s',r}p(s'|s,a)\left[r(s,a)+\gamma\underset{a}{\operatorname{max}}q_*(s',a')\right]
\end{align}
$$

## Optimality and Approximation

* In practice it is almost never possible to find the optimal values and policies
* Computation required to calculate all values, and memory required to store them, are not realistic
* Using function approximation, memory requirements can be reduced to a small set of parameters
* Online learning allows RL algorithms to focus more on states actually encountered, not wasting computation on irrelevant state values